In [2]:
!pip install -q "flwr[simulation]" "flwr-datasets[vision]" torch torchvision


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [3]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import flwr
from flwr.client import Client, ClientApp, NumPyClient
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
from flwr.server.strategy import FedAvg, FedAdagrad
from flwr.simulation import run_simulation
from flwr_datasets import FederatedDataset
from flwr.common import ndarrays_to_parameters, NDArrays, Scalar, Context

DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")

/Users/kevincorstorphine/.local/share/virtualenvs/ai.llm_prompting-rP8wfVkF/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-04 08:43:57,642	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Training on cpu
Flower 1.20.0 / PyTorch 2.2.2


In [4]:
NUM_PARTITIONS = 10
BATCH_SIZE = 32


def load_datasets(partition_id: int, num_partitions: int):
    fds = FederatedDataset(dataset="cifar10", partitioners={"train": num_partitions})
    partition = fds.load_partition(partition_id)
    # Divide data on each node: 80% train, 20% test
    partition_train_test = partition.train_test_split(test_size=0.2, seed=42)
    pytorch_transforms = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )

    def apply_transforms(batch):
        # Instead of passing transforms to CIFAR10(..., transform=transform)
        # we will use this function to dataset.with_transform(apply_transforms)
        # The transforms object is exactly the same
        batch["img"] = [pytorch_transforms(img) for img in batch["img"]]
        return batch

    partition_train_test = partition_train_test.with_transform(apply_transforms)
    trainloader = DataLoader(
        partition_train_test["train"], batch_size=BATCH_SIZE, shuffle=True
    )
    valloader = DataLoader(partition_train_test["test"], batch_size=BATCH_SIZE)
    testset = fds.load_split("test").with_transform(apply_transforms)
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    return trainloader, valloader, testloader

In [5]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for batch in trainloader:
            images, labels = batch["img"], batch["label"]
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for batch in testloader:
            images, labels = batch["img"], batch["label"]
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

In [6]:
class FlowerClient(NumPyClient):
    def __init__(self, partition_id, net, trainloader, valloader):
        self.partition_id = partition_id
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.partition_id}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.partition_id}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.partition_id}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(context: Context) -> Client:
    net = Net().to(DEVICE)

    # Read the node_config to fetch data partition associated to this node
    partition_id = context.node_config["partition-id"]
    num_partitions = context.node_config["num-partitions"]

    trainloader, valloader, _ = load_datasets(partition_id, num_partitions)
    return FlowerClient(partition_id, net, trainloader, valloader).to_client()


# Create the ClientApp
client = ClientApp(client_fn=client_fn)

In [7]:
# Create an instance of the model and get the parameters
params = get_parameters(Net())

In [8]:
def server_fn(context: Context) -> ServerAppComponents:
    # Create FedAvg strategy
    strategy = FedAvg(
        fraction_fit=0.3,
        fraction_evaluate=0.3,
        min_fit_clients=3,
        min_evaluate_clients=3,
        min_available_clients=NUM_PARTITIONS,
        initial_parameters=ndarrays_to_parameters(
            params
        ),  # Pass initial model parameters
    )

    # Configure the server for 3 rounds of training
    config = ServerConfig(num_rounds=3)
    return ServerAppComponents(strategy=strategy, config=config)

In [9]:
# Create ServerApp
server = ServerApp(server_fn=server_fn)

In [10]:
# Specify the resources each of your clients need
# If set to none, by default, each client will be allocated 2x CPU and 0x GPUs
backend_config = {"client_resources": None}
if DEVICE.type == "cuda":
    backend_config = {"client_resources": {"num_gpus": 1}}

# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_PARTITIONS,
    backend_config=backend_config,
)

INFO :      Starting Flower ServerApp, config: num_rounds=3, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
INFO :      Evaluation returned no results (`None`)
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 3 clients (out of 10)


(ClientAppActor pid=81702) [Client 2] fit, config: {}


INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


(ClientAppActor pid=81702) Epoch 1: train loss 0.06646657735109329, accuracy 0.20875
(ClientAppActor pid=81702) [Client 5] evaluate, config: {}
(ClientAppActor pid=81701) [Client 7] fit, config: {} [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)


INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 3 clients (out of 10)


(ClientAppActor pid=81701) Epoch 1: train loss 0.05938024818897247, accuracy 0.30225 [repeated 3x across cluster]
(ClientAppActor pid=81701) [Client 4] evaluate, config: {} [repeated 2x across cluster]
(ClientAppActor pid=81703) [Client 8] fit, config: {} [repeated 3x across cluster]


INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)
INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [ROUND 3]
INFO :      configure_fit: strategy sampled 3 clients (out of 10)
(ClientAppActor pid=81703) OMP: Warning #191: Forking a process while a parallel region is active is potentially unsafe.


(ClientAppActor pid=81702) Epoch 1: train loss 0.055210284888744354, accuracy 0.355 [repeated 3x across cluster]
(ClientAppActor pid=81703) [Client 2] evaluate, config: {} [repeated 3x across cluster]
(ClientAppActor pid=81701) [Client 5] fit, config: {} [repeated 3x across cluster]


INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)
INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 3 round(s) in 42.14s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.06449337780475616
INFO :      		round 2: 0.05782590349515279
INFO :      		round 3: 0.05459944880008697
INFO :      


(ClientAppActor pid=81701) Epoch 1: train loss 0.055952202528715134, accuracy 0.33725 [repeated 2x across cluster]
(ClientAppActor pid=81702) [Client 8] evaluate, config: {} [repeated 3x across cluster]


In [ ]:
def server_fn(context: Context) -> ServerAppComponents:
    # Create FedAdagrad strategy
    strategy = FedAdagrad(
        fraction_fit=0.3,
        fraction_evaluate=0.3,
        min_fit_clients=3,
        min_evaluate_clients=3,
        min_available_clients=NUM_PARTITIONS,
        initial_parameters=ndarrays_to_parameters(params),
    )
    # Configure the server for 3 rounds of training
    config = ServerConfig(num_rounds=3)
    return ServerAppComponents(strategy=strategy, config=config)


# Create the ServerApp
server = ServerApp(server_fn=server_fn)

# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_PARTITIONS,
    backend_config=backend_config,
)

In [11]:
def server_fn(context: Context) -> ServerAppComponents:
    # Create FedAdagrad strategy
    strategy = FedAdagrad(
        fraction_fit=0.3,
        fraction_evaluate=0.3,
        min_fit_clients=3,
        min_evaluate_clients=3,
        min_available_clients=NUM_PARTITIONS,
        initial_parameters=ndarrays_to_parameters(params),
    )
    # Configure the server for 3 rounds of training
    config = ServerConfig(num_rounds=3)
    return ServerAppComponents(strategy=strategy, config=config)


# Create the ServerApp
server = ServerApp(server_fn=server_fn)

# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_PARTITIONS,
    backend_config=backend_config,
)

INFO :      Starting Flower ServerApp, config: num_rounds=3, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
INFO :      Evaluation returned no results (`None`)
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 3 clients (out of 10)


(ClientAppActor pid=83183) [Client 2] fit, config: {}


(ClientAppActor pid=83182) OMP: Warning #191: Forking a process while a parallel region is active is potentially unsafe.
INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


(ClientAppActor pid=83181) Epoch 1: train loss 0.06658924371004105, accuracy 0.175
(ClientAppActor pid=83183) [Client 1] evaluate, config: {}


INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 3 clients (out of 10)


...

(ClientAppActor pid=83183) [Client 3] fit, config: {} [repeated 3x across cluster]


INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


(ClientAppActor pid=83183) Epoch 1: train loss 0.11036675423383713, accuracy 0.145 [repeated 3x across cluster]
(ClientAppActor pid=83182) [Client 4] evaluate, config: {} [repeated 3x across cluster]


INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 3 round(s) in 32.54s
INFO :      	History (loss, distributed):
INFO :      		round 1: 5.984871665954589
INFO :      		round 2: 0.6313988952636719
INFO :      		round 3: 0.29680082845687866
INFO :      


(ClientAppActor pid=83182) [Client 9] fit, config: {} [repeated 2x across cluster]
(ClientAppActor pid=83182) Epoch 1: train loss 0.10720989108085632, accuracy 0.14175 [repeated 2x across cluster]
(ClientAppActor pid=83181) [Client 2] evaluate, config: {} [repeated 2x across cluster]


In [12]:
# The `evaluate` function will be called by Flower after every round
def evaluate(
    server_round: int,
    parameters: NDArrays,
    config: Dict[str, Scalar],
) -> Optional[Tuple[float, Dict[str, Scalar]]]:
    net = Net().to(DEVICE)
    _, _, testloader = load_datasets(0, NUM_PARTITIONS)
    set_parameters(net, parameters)  # Update model with the latest parameters
    loss, accuracy = test(net, testloader)
    print(f"Server-side evaluation loss {loss} / accuracy {accuracy}")
    return loss, {"accuracy": accuracy}

In [13]:
def server_fn(context: Context) -> ServerAppComponents:
    # Create the FedAvg strategy
    strategy = FedAvg(
        fraction_fit=0.3,
        fraction_evaluate=0.3,
        min_fit_clients=3,
        min_evaluate_clients=3,
        min_available_clients=NUM_PARTITIONS,
        initial_parameters=ndarrays_to_parameters(params),
        evaluate_fn=evaluate,  # Pass the evaluation function
    )
    # Configure the server for 3 rounds of training
    config = ServerConfig(num_rounds=3)
    return ServerAppComponents(strategy=strategy, config=config)


# Create the ServerApp
server = ServerApp(server_fn=server_fn)

In [14]:
# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_PARTITIONS,
    backend_config=backend_config,
)

INFO :      Starting Flower ServerApp, config: num_rounds=3, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
INFO :      initial parameters (loss, other metrics): 0.07212381489276885, {'accuracy': 0.1}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.07212381489276885 / accuracy 0.1
(ClientAppActor pid=84725) [Client 1] fit, config: {}


INFO :      aggregate_fit: received 3 results and 0 failures


(ClientAppActor pid=84725) Epoch 1: train loss 0.06733594089746475, accuracy 0.183


INFO :      fit progress: (1, 0.06314338445663452, {'accuracy': 0.285}, 15.924109970015706)
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


...

Server-side evaluation loss 0.05485567938089371 / accuracy 0.3582
(ClientAppActor pid=84724) [Client 7] evaluate, config: {} [repeated 3x across cluster]
(ClientAppActor pid=84723) [Client 0] fit, config: {} [repeated 3x across cluster]
(ClientAppActor pid=84723) Epoch 1: train loss 0.059288717806339264, accuracy 0.31175 [repeated 3x across cluster]


INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [ROUND 3]
INFO :      configure_fit: strategy sampled 3 clients (out of 10)
INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      fit progress: (3, 0.05179705816507339, {'accuracy': 0.3893}, 40.89951680100057)
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.05179705816507339 / accuracy 0.3893
(ClientAppActor pid=84725) [Client 3] evaluate, config: {} [repeated 3x across cluster]
(ClientAppActor pid=84724) [Client 9] fit, config: {} [repeated 3x across cluster]
(ClientAppActor pid=84724) Epoch 1: train loss 0.05409356951713562, accuracy 0.36675 [repeated 3x across cluster]


(ClientAppActor pid=84724) OMP: Warning #191: Forking a process while a parallel region is active is potentially unsafe.
INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 3 round(s) in 43.08s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.06494624384244284
INFO :      		round 2: 0.056942435979843144
INFO :      		round 3: 0.05410506474971771
INFO :      	History (loss, centralized):
INFO :      		round 0: 0.07212381489276885
INFO :      		round 1: 0.06314338445663452
INFO :      		round 2: 0.05485567938089371
INFO :      		round 3: 0.05179705816507339
INFO :      	History (metrics, centralized):
INFO :      	{'accuracy': [(0, 0.1), (1, 0.285), (2, 0.3582), (3, 0.3893)]}
INFO :      


(ClientAppActor pid=84724) [Client 2] evaluate, config: {} [repeated 2x across cluster]


In [15]:
class FlowerClient(NumPyClient):
    def __init__(self, pid, net, trainloader, valloader):
        self.pid = pid  # partition ID of a client
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.pid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        # Read values from config
        server_round = config["server_round"]
        local_epochs = config["local_epochs"]

        # Use values provided by the config
        print(f"[Client {self.pid}, round {server_round}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=local_epochs)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.pid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(context: Context) -> Client:
    net = Net().to(DEVICE)
    partition_id = context.node_config["partition-id"]
    num_partitions = context.node_config["num-partitions"]
    trainloader, valloader, _ = load_datasets(partition_id, num_partitions)
    return FlowerClient(partition_id, net, trainloader, valloader).to_client()


# Create the ClientApp
client = ClientApp(client_fn=client_fn)

In [16]:
def fit_config(server_round: int):
    """Return training configuration dict for each round.

    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        "server_round": server_round,  # The current round of federated learning
        "local_epochs": 1 if server_round < 2 else 2,
    }
    return config

In [17]:
def server_fn(context: Context) -> ServerAppComponents:
    # Create FedAvg strategy
    strategy = FedAvg(
        fraction_fit=0.3,
        fraction_evaluate=0.3,
        min_fit_clients=3,
        min_evaluate_clients=3,
        min_available_clients=NUM_PARTITIONS,
        initial_parameters=ndarrays_to_parameters(params),
        evaluate_fn=evaluate,
        on_fit_config_fn=fit_config,  # Pass the fit_config function
    )
    config = ServerConfig(num_rounds=3)
    return ServerAppComponents(strategy=strategy, config=config)


# Create the ServerApp
server = ServerApp(server_fn=server_fn)

# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_PARTITIONS,
    backend_config=backend_config,
)

INFO :      Starting Flower ServerApp, config: num_rounds=3, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
INFO :      initial parameters (loss, other metrics): 0.07212381489276885, {'accuracy': 0.1}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.07212381489276885 / accuracy 0.1
(ClientAppActor pid=86303) [Client 5, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}
(ClientAppActor pid=86303) Epoch 1: train loss 0.06666231900453568, accuracy 0.192


INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      fit progress: (1, 0.06314804443120957, {'accuracy': 0.2769}, 15.443437120004091)
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.06314804443120957 / accuracy 0.2769
(ClientAppActor pid=86304) [Client 0] evaluate, config: {}
(ClientAppActor pid=86304) [Client 0, round 1] fit, config: {'server_round': 1, 'local_epochs': 1} [repeated 2x across cluster]
(ClientAppActor pid=86304) Epoch 1: train loss 0.06663762778043747, accuracy 0.20425 [repeated 2x across cluster]


INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 3 clients (out of 10)
(ClientAppActor pid=86304) OMP: Warning #191: Forking a process while a parallel region is active is potentially unsafe.
INFO :      aggregate_fit: received 3 results and 0 failures


...

(ClientAppActor pid=86304) Epoch 2: train loss 0.049270663410425186, accuracy 0.42175 [repeated 4x across cluster]
(ClientAppActor pid=86302) [Client 5] evaluate, config: {}
(ClientAppActor pid=86302) [Client 1, round 3] fit, config: {'server_round': 3, 'local_epochs': 2} [repeated 2x across cluster]


INFO :      fit progress: (3, 0.04829498074054718, {'accuracy': 0.4294}, 44.07115524201072)
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.04829498074054718 / accuracy 0.4294
(ClientAppActor pid=86303) [Client 0] evaluate, config: {}
(ClientAppActor pid=86302) Epoch 2: train loss 0.050026677548885345, accuracy 0.41025 [repeated 2x across cluster]


INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 3 round(s) in 46.25s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.06468521841367086
INFO :      		round 2: 0.05344654881954194
INFO :      		round 3: 0.04934217663606008
INFO :      	History (loss, centralized):
INFO :      		round 0: 0.07212381489276885
INFO :      		round 1: 0.06314804443120957
INFO :      		round 2: 0.051997402584552765
INFO :      		round 3: 0.04829498074054718
INFO :      	History (metrics, centralized):
INFO :      	{'accuracy': [(0, 0.1), (1, 0.2769), (2, 0.3903), (3, 0.4294)]}
INFO :      


(ClientAppActor pid=86304) [Client 3] evaluate, config: {} [repeated 2x across cluster]


In [18]:
class FlowerClient(NumPyClient):
    def __init__(self, pid, net, trainloader, valloader):
        self.pid = pid  # partition ID of a client
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.pid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        # Read values from config
        server_round = config["server_round"]
        local_epochs = config["local_epochs"]

        # Use values provided by the config
        print(f"[Client {self.pid}, round {server_round}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=local_epochs)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.pid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(context: Context) -> Client:
    net = Net().to(DEVICE)
    partition_id = context.node_config["partition-id"]
    num_partitions = context.node_config["num-partitions"]
    trainloader, valloader, _ = load_datasets(partition_id, num_partitions)
    return FlowerClient(partition_id, net, trainloader, valloader).to_client()


# Create the ClientApp
client = ClientApp(client_fn=client_fn)

In [19]:
def fit_config(server_round: int):
    """Return training configuration dict for each round.

    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        "server_round": server_round,  # The current round of federated learning
        "local_epochs": 1 if server_round < 2 else 2,
    }
    return config

In [20]:
def server_fn(context: Context) -> ServerAppComponents:
    # Create FedAvg strategy
    strategy = FedAvg(
        fraction_fit=0.3,
        fraction_evaluate=0.3,
        min_fit_clients=3,
        min_evaluate_clients=3,
        min_available_clients=NUM_PARTITIONS,
        initial_parameters=ndarrays_to_parameters(params),
        evaluate_fn=evaluate,
        on_fit_config_fn=fit_config,  # Pass the fit_config function
    )
    config = ServerConfig(num_rounds=3)
    return ServerAppComponents(strategy=strategy, config=config)


# Create the ServerApp
server = ServerApp(server_fn=server_fn)

# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_PARTITIONS,
    backend_config=backend_config,
)

INFO :      Starting Flower ServerApp, config: num_rounds=3, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
INFO :      initial parameters (loss, other metrics): 0.07212381489276885, {'accuracy': 0.1}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.07212381489276885 / accuracy 0.1
(ClientAppActor pid=88983) [Client 2, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}


(ClientAppActor pid=88984) OMP: Warning #191: Forking a process while a parallel region is active is potentially unsafe.
INFO :      aggregate_fit: received 3 results and 0 failures


(ClientAppActor pid=88983) Epoch 1: train loss 0.06749646365642548, accuracy 0.1845


INFO :      fit progress: (1, 0.06328895518779755, {'accuracy': 0.2796}, 15.689470037003048)
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


...

INFO :      fit progress: (3, 0.04950836522579193, {'accuracy': 0.4202}, 47.20358237600885)
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.04950836522579193 / accuracy 0.4202
(ClientAppActor pid=88984) [Client 2] evaluate, config: {}
(ClientAppActor pid=88982) Epoch 2: train loss 0.0500611886382103, accuracy 0.424 [repeated 3x across cluster]


INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 3 round(s) in 49.39s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.06507738137245178
INFO :      		round 2: 0.0546703929901123
INFO :      		round 3: 0.050372700770696
INFO :      	History (loss, centralized):
INFO :      		round 0: 0.07212381489276885
INFO :      		round 1: 0.06328895518779755
INFO :      		round 2: 0.05266948276758194
INFO :      		round 3: 0.04950836522579193
INFO :      	History (metrics, centralized):
INFO :      	{'accuracy': [(0, 0.1), (1, 0.2796), (2, 0.3845), (3, 0.4202)]}
INFO :      


(ClientAppActor pid=88982) [Client 0] evaluate, config: {} [repeated 2x across cluster]


(ClientAppActor pid=88982) OMP: Warning #191: Forking a process while a parallel region is active is potentially unsafe.


In [21]:
NUM_PARTITIONS = 1000

In [22]:
def fit_config(server_round: int):
    config = {
        "server_round": server_round,
        "local_epochs": 3,
    }
    return config


def server_fn(context: Context) -> ServerAppComponents:
    # Create FedAvg strategy
    strategy = FedAvg(
        fraction_fit=0.025,  # Train on 25 clients (each round)
        fraction_evaluate=0.05,  # Evaluate on 50 clients (each round)
        min_fit_clients=20,
        min_evaluate_clients=40,
        min_available_clients=NUM_PARTITIONS,
        initial_parameters=ndarrays_to_parameters(params),
        on_fit_config_fn=fit_config,
    )
    config = ServerConfig(num_rounds=3)
    return ServerAppComponents(strategy=strategy, config=config)


# Create the ServerApp
server = ServerApp(server_fn=server_fn)

# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_PARTITIONS,
    backend_config=backend_config,
)

INFO :      Starting Flower ServerApp, config: num_rounds=3, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
INFO :      Evaluation returned no results (`None`)
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 20 clients (out of 1000)


(ClientAppActor pid=94819) [Client 352, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(ClientAppActor pid=94819) Epoch 1: train loss 0.11568943411111832, accuracy 0.075
(ClientAppActor pid=94819) Epoch 2: train loss 0.11468267440795898, accuracy 0.075


INFO :      aggregate_fit: received 20 results and 0 failures
INFO :      configure_evaluate: strategy sampled 50 clients (out of 1000)


(ClientAppActor pid=94823) [Client 9] evaluate, config: {}
(ClientAppActor pid=94823) [Client 405, round 1] fit, config: {'server_round': 1, 'local_epochs': 3} [repeated 19x across cluster]
(ClientAppActor pid=94823) Epoch 3: train loss 0.11450189352035522, accuracy 0.1 [repeated 58x across cluster]
(ClientAppActor pid=94823) [Client 651] evaluate, config: {} [repeated 32x across cluster]


INFO :      aggregate_evaluate: received 50 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 25 clients (out of 1000)


...

(ClientAppActor pid=94819) HTTP Error 429 thrown while requesting HEAD https://huggingface.co/datasets/cifar10/resolve/main/README.md [repeated 32x across cluster]
(ClientAppActor pid=94819) Retrying in 8s [Retry 4/5]. [repeated 25x across cluster]
(ClientAppActor pid=94819) HTTP Error 429 thrown while requesting HEAD https://huggingface.co/datasets/cifar10/resolve/main/README.md [repeated 8x across cluster]
(ClientAppActor pid=94819) Retrying in 8s [Retry 5/5]. [repeated 8x across cluster]
(ClientAppActor pid=94819) HTTP Error 429 thrown while requesting HEAD https://huggingface.co/datasets/cifar10/resolve/0b2714987fa478483af9968de7c934580d0bb9a2/cifar10.py [repeated 9x across cluster]
(ClientAppActor pid=94819) Retrying in 1s [Retry 1/5]. [repeated 8x across cluster]


(ClientAppActor pid=94817) [Client 937] evaluate, config: {} [repeated 5x across cluster]
(ClientAppActor pid=94819) [Client 5] evaluate, config: {} [repeated 29x across cluster]


INFO :      aggregate_evaluate: received 50 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 3 round(s) in 206.92s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.23039207649230964
INFO :      		round 2: 0.23011634254455568
INFO :      		round 3: 0.22934848690032958
INFO :      


(ClientAppActor pid=94820) [Client 738] evaluate, config: {} [repeated 4x across cluster]


(ClientAppActor pid=94823) HTTP Error 429 thrown while requesting HEAD https://huggingface.co/datasets/cifar10/resolve/0b2714987fa478483af9968de7c934580d0bb9a2/cifar10.py [repeated 6x across cluster]
(ClientAppActor pid=94823) Retrying in 1s [Retry 1/5]. [repeated 2x across cluster]
